In [ ]:
import radiomics
import logging
import os
import pandas as pd
from pathlib import Path
from tqdm import tqdm

In [ ]:
def get_im_path(i):
    case = f"{i}".zfill(5)
    path = f"insert_path_here\\case_{case}\\imaging.nii.gz"
    return path

def get_seg_path(i):
    case = f"{i}".zfill(5)
    path = f"insert_path_here\\case_{case}\\segmentation.nii.gz"
    return path

In [ ]:
params_path = "params.yml"

In [ ]:
log_file = 'log_file.txt'
handler = logging.FileHandler(filename=log_file, mode='w')  # overwrites log_files from previous runs. Change mode to 'a' to append.
formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")  # format string for log messages
handler.setFormatter(formatter)
radiomics.logger.addHandler(handler)

# Control the amount of logging stored by setting the level of the logger. N.B. if the level is higher than the
# Verbositiy level, the logger level will also determine the amount of information printed to the output
radiomics.logger.setLevel(logging.DEBUG)


In [ ]:
extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(params_path)

In [ ]:
def extract_feature(extractor ,image_path, mask_path):
    features = dict(extractor.execute(image_path, mask_path))
    features = {k: v for k, v in features.items() if not "diagnostic" in k}
    return features

In [ ]:
for i in tqdm(range(0,212,1)): #resampled to 1x1x1 mm
#for i in tqdm(ccrccTNMdata.index, total=len(ccrccTNMdata.index)):
    try:
        image_path = get_im_path(i)
        mask_path =  get_seg_path(i)

        features = extract_feature(extractor, image_path, mask_path)
        features["case"] = f"case{i}"

        if i == 0:
            df = pd.DataFrame([features])
        else:
            df = df.append(pd.DataFrame([features]), ignore_index=True)
        
    except:
        print(i)
df.to_csv("features_df.csv", index=False)